# First look into data

In [123]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sc

Let's take a first look at our data, which consists of:
* data/obesity.csv
* data/employment.csv

In [2]:
obesity = pd.read_csv('data/obesity.csv', index_col=0)
employment = pd.read_csv('data/employment.csv')

In [3]:
# Continent function
import pycountry_convert as pc

continents_dict = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Oceania',
    'AF': 'Africa',
    'EU': 'Europe',
    '?': 'Unknown'
}

def convert_country_to_continent(country_name):
    try:
        country_code = pc.country_name_to_country_alpha2(country_name, cn_name_format="default")
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        return continents_dict[continent_code]
    except:
        
        # Exception in employment
        if country_name in ['Korea', 'Republic of Korea', 'Timor-Leste']: return continents_dict['AS']
        
        # Exceptions in obesity
        else: return convert_country_to_continent(country_name.split(" ")[0])
        
        return '?'
    

## Obesity
Data from: https://apps.who.int/gho/data/node.main.A900A?lang=en

In [4]:
obesity.head()

,Country,Year,Obesity (%),Sex
0,Afghanistan,1975,0.5 [0.2-1.1],Both sexes
1,Afghanistan,1975,0.2 [0.0-0.6],Male
2,Afghanistan,1975,0.8 [0.2-2.0],Female
3,Afghanistan,1976,0.5 [0.2-1.1],Both sexes
4,Afghanistan,1976,0.2 [0.0-0.7],Male


In [5]:
obesity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24570 entries, 0 to 24569
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Country      24570 non-null  object
 1   Year         24570 non-null  int64 
 2   Obesity (%)  24570 non-null  object
 3   Sex          24570 non-null  object
dtypes: int64(1), object(3)
memory usage: 959.8+ KB


### Variables
* **Country**: Name of a country (Afghanistan, Belgium, ...)

* **Year**: A year as an integer (1975, 1976, ...)

* **Obesity (%)**: Percentage of obesity among the population as a string like this "%f \[%f-%f\]", with *%f* a float

* **Sex**: A string representing either Males ("Male"), Females ("Female") and Both ("Both sexes")

In [6]:
# New names
names = {
    "Country" : "country",
    "Year" : "year",
    "Obesity (%)" : "str_obesity",
    "Sex" : "sex"
}

# Renaming columns
obesity = obesity.rename(columns=names)

In [7]:
obesity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24570 entries, 0 to 24569
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   country      24570 non-null  object
 1   year         24570 non-null  int64 
 2   str_obesity  24570 non-null  object
 3   sex          24570 non-null  object
dtypes: int64(1), object(3)
memory usage: 959.8+ KB


#### Obesity
The variable "obesity" must be processed because a float is wanted instead of a string as we currently have. So let's extract the 3 values in the string and create 3 variable/columns, named:
* obesity
* min_obesity
* max_obesity

In [8]:
import re # Module for regular expressions

def extract_float(string, index=0):
    """
    >>> extract_float("0.5 [0.1-0.7]")
    0.5
    """

    # Worst case
    if index < 0:
        return None

    # No data case
    if string == "No data":
        return 0.0

    # Defining the regular expression
    reg_exp = re.compile("\d+\.\d+")

    # Tuple normally containing 3 float
    t = [float(e) for e in reg_exp.findall(string)] 

    # Bad case 27 <= len(string) <= 31
    if len(string) in range(27,31):
        return (t[index] + t[index + 3]) / 2
    # Normal case, 13 <= len(string) <= 16
    else:
        return t[index]


In [9]:
# New variables
new_obesity_columns = ['obesity', 'min_obesity', 'max_obesity']

# Create the new columns
for i in range(len(new_obesity_columns)):
    obesity[new_obesity_columns[i]] = obesity['str_obesity'].apply(lambda x: extract_float(x, index=i))

In [10]:
obesity.head()

,country,year,str_obesity,sex,obesity,min_obesity,max_obesity
0,Afghanistan,1975,0.5 [0.2-1.1],Both sexes,0.5,0.2,1.1
1,Afghanistan,1975,0.2 [0.0-0.6],Male,0.2,0.0,0.6
2,Afghanistan,1975,0.8 [0.2-2.0],Female,0.8,0.2,2.0
3,Afghanistan,1976,0.5 [0.2-1.1],Both sexes,0.5,0.2,1.1
4,Afghanistan,1976,0.2 [0.0-0.7],Male,0.2,0.0,0.7


#### Sex
This variable is categorical and is representend by the following values:
* **Male**
* **Female**
* **Both sexes**

So in order to have a better visibility we will change the values by:
* Male: **M**
* Female: **F**
* Both sexes: **B**

In [11]:
# Unique values of the 'sex' variable
current_sex = set(obesity.sex)

# Create the dict in order to replace values in dataframee
new_sex = {e:e[0].upper() for e in current_sex}

# Replace values in dataframe
obesity['sex'] = obesity['sex'].replace(new_sex)

# Convert type to categorical
obesity['sex'] = obesity['sex'].astype('category')

#### Country and Year
the variable 'year' already has the right type and 'country' just need the right type.

In [12]:
# Convert type to string
obesity['country'] = obesity['country'].astype('string')

#### Some cleaning bro

In [13]:
# Keep what we want
dirty = ['str_obesity']
obesity = obesity[ [e for e in obesity.columns if e not in dirty] ]

#### Continent
The variable 'continent' will be added because of grouping per continent.

In [14]:
obesity['continent'] = obesity['country'].apply(lambda x: convert_country_to_continent(x))

### Final result

In [15]:
obesity.head()

,country,year,sex,obesity,min_obesity,max_obesity,continent
0,Afghanistan,1975,B,0.5,0.2,1.1,Asia
1,Afghanistan,1975,M,0.2,0.0,0.6,Asia
2,Afghanistan,1975,F,0.8,0.2,2.0,Asia
3,Afghanistan,1976,B,0.5,0.2,1.1,Asia
4,Afghanistan,1976,M,0.2,0.0,0.7,Asia


In [16]:
obesity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24570 entries, 0 to 24569
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   country      24570 non-null  string  
 1   year         24570 non-null  int64   
 2   sex          24570 non-null  category
 3   obesity      24570 non-null  float64 
 4   min_obesity  24570 non-null  float64 
 5   max_obesity  24570 non-null  float64 
 6   continent    24570 non-null  object  
dtypes: category(1), float64(3), int64(1), object(1), string(1)
memory usage: 1.3+ MB


In [17]:
obesity.describe()

,year,obesity,min_obesity,max_obesity
count,24570.000000,24570.000000,24570.000000,24570.000000
mean,1995.500000,12.193630,9.047705,15.899422
std,12.121165,10.450249,8.860355,12.100114
min,1975.000000,0.000000,0.000000,0.000000
25%,1985.000000,3.600000,2.000000,5.800000
50%,1995.500000,10.300000,6.800000,14.500000
75%,2006.000000,18.000000,13.600000,22.900000
max,2016.000000,63.300000,55.600000,70.800000


### Save the data

In [18]:
# obesity.to_csv('data/obesity_final.csv', index=False)

## Employment
Data from: https://stats.oecd.org/Index.aspx?DataSetCode=QNA
Theme: **Labour/Labour Force Statistics/Annual Labour Force Statistics/Employment by activities**

In [19]:
employment.head(2)

,LOCATION,Country,SUBJECT,Subject,SEX,Sex,FREQUENCY,Frequency,TIME,Time,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,BEL,Belgium,YA994TL1_ST,"Employment in all activities (ISIC rev.4, A-U)",MA,Males,A,Annual,2008,2008,PER,Persons,3,Thousands,NaN,NaN,2460.675,NaN,NaN
1,BEL,Belgium,YA994TL1_ST,"Employment in all activities (ISIC rev.4, A-U)",MA,Males,A,Annual,2009,2009,PER,Persons,3,Thousands,NaN,NaN,2429.325,NaN,NaN


In [20]:
employment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24880 entries, 0 to 24879
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   LOCATION               24880 non-null  object 
 1   Country                24880 non-null  object 
 2   SUBJECT                24880 non-null  object 
 3   Subject                24880 non-null  object 
 4   SEX                    24880 non-null  object 
 5   Sex                    24880 non-null  object 
 6   FREQUENCY              24880 non-null  object 
 7   Frequency              24880 non-null  object 
 8   TIME                   24880 non-null  int64  
 9   Time                   24880 non-null  int64  
 10  Unit Code              24880 non-null  object 
 11  Unit                   24880 non-null  object 
 12  PowerCode Code         24880 non-null  int64  
 13  PowerCode              24880 non-null  object 
 14  Reference Period Code  0 non-null      float64
 15  Re

### Variables

Here are the most relevant variables that will be needed for the project:
* **LOCATION**: The country code, in case we need it
* **Country**: The name of the country
* **Subject**: The type of professional activity
* **Sex**: The gender
* **Time**: The year 
* **Value**: The number of people times the 10^**PowerCode**

#### Dropping the useless columns

In [21]:
useless_columns = ['SUBJECT', 'Frequency', 'SEX', 'FREQUENCY', 'TIME', 'Unit Code', 'Reference Period Code', 'Reference Period', 'Flag Codes', 'Flags', 'Unit', 'PowerCode', 'PowerCode Code']
employment = employment.drop(columns=useless_columns)

#### LOCATION & Country
Let's rename the indexes and change the type of the variables in order to make them more understandable.

In [22]:
loc_country = {'LOCATION':'country_code', 'Country':'country'}
employment = employment.rename(columns=loc_country)

for col in loc_country.values():
    employment[col] = employment[col].astype('string')

#### Subject
Let's try to understand this variable and make it more understandable.

In [23]:
employment['Subject'].unique().tolist()


['Employment in all activities (ISIC rev.4, A-U)',
 'Employment in Agriculture,hunting and forestry (ISIC rev4, A)',
 'Agriculture,hunting and forestry (A) ',
 'Employment in Industry (ISIC rev4, B-F)',
 'Mining and quarrying (B)',
 'Manufacturing (C )',
 'Electricity, gas, steam and air conditioning supply (D)',
 'Water supply, sewerage, waste management and remediation activities (E)',
 'Construction (F)',
 'Employment in Services (ISIC rev.4, G-U)',
 'Wholesale and retail trade, repair of motor vehicles and motorcycles (G)',
 'Transportation and storage (H)',
 'Accommodation and food service activities (I)',
 'Information and communication (J)',
 'Financial and insurance activities (K)',
 'Real estate activities (L)',
 'Professional, scientific and technical activities (M)',
 'Administrative and support service activities (N)',
 'Public administration and defence, Compulsory social security (O)',
 'Education (P)',
 'Human health and social work activities (Q)',
 'Arts, entertainment

#### Task: Identify which activity can be classified as desk job.
After identifying the manual and desk activities we will create a new variable 'activity' which will either be **MANUAL** or **DESK**.

In [24]:
# Temporary identification
manual_activity = [
    'Employment in Agriculture,hunting and forestry (ISIC rev4, A)', # + Ss categories
    'Employment in Industry (ISIC rev4, B-F)', # + Ss categories
    'Wholesale and retail trade, repair of motor vehicles and motorcycles (G)',
    'Transportation and storage (H)',
    'Accommodation and food service activities (I)',
    'Other service activities (S)',
    'Activities of households as employers undifferentiated goods- and services-producing activities of households for own use (T)',
]

no_manual_activity = [
    'Information and communication (J)',
    'Financial and insurance activities (K)',
    'Real estate activities (L)',
    'Professional, scientific and technical activities (M)',
    'Administrative and support service activities (N)',
    'Public administration and defence, Compulsory social security (O)',
    'Education (P)',
    'Human health and social work activities (Q)',
    'Arts, entertainment and recreation ( R)',
    'Activities of extraterritorial organisations and bodies (U)'
]

def is_desk_manual(string):
    manual = ['A', 'B', 'C ', 'D', 'E', 'F', 'G', 'H', 'I', 'S', 'T']
    desk = ['J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', ' R', 'U']
    
    for x in manual:
        if '({})'.format(x) in string: return 'M'
    for x in desk:
        if '({})'.format(x) in string: return 'D'
        
    return 'U'

    

In [25]:
# Create a new variable
employment['activity'] = employment['Subject'].apply(lambda x: is_desk_manual(x))

# Delete the former
employment = employment.rename(columns={'Subject':'subject'})

In [26]:
# Change the type of the variable
employment['activity'] = employment['activity'].astype('category')

In [27]:
employment.query("activity == 'D'")['subject'].unique().tolist()

['Information and communication (J)',
 'Financial and insurance activities (K)',
 'Real estate activities (L)',
 'Professional, scientific and technical activities (M)',
 'Administrative and support service activities (N)',
 'Public administration and defence, Compulsory social security (O)',
 'Education (P)',
 'Human health and social work activities (Q)',
 'Arts, entertainment and recreation ( R)',
 'Activities of extraterritorial organisations and bodies (U)']

In [28]:
employment.query("activity == 'M'")['subject'].unique().tolist()

['Agriculture,hunting and forestry (A) ',
 'Mining and quarrying (B)',
 'Manufacturing (C )',
 'Electricity, gas, steam and air conditioning supply (D)',
 'Water supply, sewerage, waste management and remediation activities (E)',
 'Construction (F)',
 'Wholesale and retail trade, repair of motor vehicles and motorcycles (G)',
 'Transportation and storage (H)',
 'Accommodation and food service activities (I)',
 'Other service activities (S)',
 'Activities of households as employers undifferentiated goods- and services-producing activities of households for own use (T)']

In [29]:
employment.query("activity == 'U'")['subject'].unique().tolist()

['Employment in all activities (ISIC rev.4, A-U)',
 'Employment in Agriculture,hunting and forestry (ISIC rev4, A)',
 'Employment in Industry (ISIC rev4, B-F)',
 'Employment in Services (ISIC rev.4, G-U)']

#### Sex
Let's change values in order to make them like in the **obesity dataset** and also change the type.

In [30]:
employment['Sex'].unique().tolist()

['Males', 'Females', 'All persons']

In [31]:
sex_values = {
    "Males":'M',
    "Females":'F',
    "All persons":'B'
}

# Replace values
employment = employment.replace({'Sex':sex_values})

# Change the type of the variable
employment['Sex'] = employment['Sex'].astype('category')

# Rename variable
employment = employment.rename(columns={'Sex':'sex'})

#### Time
Let's take a look at this variable.

In [32]:
employment['Time'].value_counts()

2016    2179
2017    2165
2018    2162
2015    2135
2013    2116
2014    2112
2019    2080
2012    2063
2011    2050
2010    2046
2009    1917
2008    1855
Name: Time, dtype: int64

We can see that the **Time** variable represents years between \[2008:2019\]. So let's replace the **Time** variable to **year**.

In [33]:
# Change variable
employment = employment.rename(columns={'Time':'year'})

#### Value
The most important variable.

In [34]:
# Rename the variable
employment = employment.rename(columns={'Value':'value'})

# Changing the values
employment['value'] = employment['value'].apply(lambda x: x* 1000) 

#### Continent
The 'continent' variable is also added.

In [35]:
employment['continent'] = employment['country'].apply(lambda x: convert_country_to_continent(x))

### Final Check

In [36]:
employment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24880 entries, 0 to 24879
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   country_code  24880 non-null  string  
 1   country       24880 non-null  string  
 2   subject       24880 non-null  object  
 3   sex           24880 non-null  category
 4   year          24880 non-null  int64   
 5   value         24880 non-null  float64 
 6   activity      24880 non-null  category
 7   continent     24880 non-null  object  
dtypes: category(2), float64(1), int64(1), object(2), string(2)
memory usage: 1.2+ MB


In [37]:
cat = ['sex', 'activity']

for e in cat:
    print(employment[e].value_counts())
    print()

B    10067
M     7411
F     7402
Name: sex, dtype: int64

M    10750
D     9643
U     4487
Name: activity, dtype: int64



In [38]:
employment['country'].unique()

<StringArray>
[        'Belgium',  'Czech Republic',         'Denmark',         'Finland',
         'Germany',          'Greece',         'Hungary',         'Iceland',
           'Italy',           'Japan',           'Korea',          'Mexico',
     'Netherlands',          'Norway',          'Poland',        'Portugal',
 'Slovak Republic',           'Spain',          'Sweden',     'Switzerland',
          'Turkey',  'United Kingdom',         'Estonia',        'Slovenia',
     'New Zealand',      'Luxembourg',         'Austria',          'Israel',
          'Latvia',          'France',         'Ireland',       'Australia',
           'Chile',       'Lithuania',          'Brazil',      'Costa Rica',
          'Russia',        'Colombia']
Length: 38, dtype: string

In [39]:
employment.describe()

,year,value
count,24880.000000,2.488000e+04
mean,2013.628014,1.358108e+06
std,3.412644,4.635936e+06
min,2008.000000,0.000000e+00
25%,2011.000000,3.945000e+04
50%,2014.000000,1.700875e+05
75%,2017.000000,7.964125e+05
max,2019.000000,9.220573e+07


### Save the data

In [40]:
# employment.to_csv('data/employment_final.csv', index=False)

# Merge into one DataFame

In [41]:
obesity.head(3)

,country,year,sex,obesity,min_obesity,max_obesity,continent
0,Afghanistan,1975,B,0.5,0.2,1.1,Asia
1,Afghanistan,1975,M,0.2,0.0,0.6,Asia
2,Afghanistan,1975,F,0.8,0.2,2.0,Asia


In [42]:
employment.head(3)

,country_code,country,subject,sex,year,value,activity,continent
0,BEL,Belgium,"Employment in all activities (ISIC rev.4, A-U)",M,2008,2460675.0,U,Europe
1,BEL,Belgium,"Employment in all activities (ISIC rev.4, A-U)",M,2009,2429325.0,U,Europe
2,BEL,Belgium,"Employment in all activities (ISIC rev.4, A-U)",M,2010,2457775.0,U,Europe


In [52]:
print('Dimensions obesity: ' + str(obesity.shape))
print('Dimensions employment: ' + str(employment.shape))

Dimensions obesity: (24570, 7)
Dimensions employment: (24880, 8)


In [231]:
analytics = pd.merge(obesity, employment, on=['country', 'year', 'sex', 'continent'])
analytics.head()

,country,year,sex,obesity,min_obesity,max_obesity,continent,country_code,subject,value,activity
0,Australia,2008,B,24.6,22.1,27.2,Oceania,AUS,"Employment in all activities (ISIC rev.4, A-U)",10695120.0,U
1,Australia,2008,B,24.6,22.1,27.2,Oceania,AUS,"Employment in Agriculture,hunting and forestry...",344739.0,U
2,Australia,2008,B,24.6,22.1,27.2,Oceania,AUS,"Employment in Services (ISIC rev.4, G-U)",7954637.0,U
3,Australia,2008,B,24.6,22.1,27.2,Oceania,AUS,"Employment in Industry (ISIC rev4, B-F)",2395745.0,U
4,Australia,2008,B,24.6,22.1,27.2,Oceania,AUS,Other service activities (S),233830.0,M


In [239]:
import plotly.express as px

# Filtrage + nettoyage
df = analytics.query("activity == 'M' and sex == 'B' and country == 'France'").copy()
df = df.drop(["country_code", "min_obesity", "max_obesity", "sex", "continent", "country"], axis=1)

# Recuperation des chiffres de l'emploi par activite
df['value_mean'] = df.groupby(["year", "activity"]).value.transform('mean')

# Nettoyage
df = df.drop(["subject", "value"], axis=1).drop_duplicates()

trend = "ols" if False else "lowess"

px.scatter(df, x='obesity', y='value_mean', trendline=trend)